<a href="https://colab.research.google.com/github/anastasiiaCher/python-geo-viz/blob/anastasiiaCher-patch-1/PythonUrban_final_task_Nevskiy_District.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговый проект. Улицы. Объекты культурного наследия.

In [2]:
# TODO собрать установку всех необходимых модулей в одном месте
%%capture
!pip install geopandas mapclassify  # mapclassify для визуализации
!pip install osmnx  # устанавливаем модуль osmnx
!pip install keplergl  # для визуализации

In [80]:
# TODO собрать импорты всех модулей в одном месте
import geopandas as gpd
from keplergl import KeplerGl
import numpy as np
import osmnx as ox
import pandas as pd
import re

from google.colab import output
output.enable_custom_widget_manager()

In [18]:
TILES = "CartoDB positron"  # Название подложки для карт

# TODO указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = "Невский район, Санкт-Петербург"  # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/anastasiiaCher/python-geo-viz/main/geojson_layers/kgiop_objects.geojson"  # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL = "https://raw.githubusercontent.com/anastasiiaCher/python-geo-viz/main/geojson_layers/streets.geojson"  # ссылка на слой с улицами

## Территория

### Загрузка территории из OSM (Extract)

In [117]:
# TODO загрузить geodataframe с геометрией для территории TERRITORY_NAME

district = ox.geocode_to_gdf(TERRITORY_NAME)
display(district.T)
district.explore(height=400)

,0
geometry,"POLYGON ((30.3633101 59.9138661, 30.3634988 59..."
bbox_north,59.93694
bbox_south,59.807772
bbox_east,30.567166
bbox_west,30.36331
place_id,181782309
osm_type,relation
osm_id,368287
lat,59.870794
lon,30.466033


## Улицы

### Загрузка файла с улицами (Extract)

In [118]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
gdf_streets = gpd.read_file(STREETS_FILE_URL, mask=district)
display(gdf_streets.shape)
display(gdf_streets.head(3))
display(gdf_streets.explore(tiles=TILES))

(1287, 2)

,name,geometry
0,улица Чернова,"LINESTRING (3391641.514 8369024.600, 3391686.0..."
1,Зольная улица,"LINESTRING (3388065.876 8383198.675, 3388058.2..."
2,Российский проспект,"LINESTRING (3391979.892 8382039.064, 3391986.1..."


### Обработка данных с улицами (Transform)

In [119]:
# TODO сгруппировать и объединить геометрии с одинаковыми названиями
streets = gdf_streets.dissolve(by="name")
streets.explore(tiles=TILES)

In [120]:
display(streets.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 177 entries, 1-й Рабфаковский переулок to улица Юннатов
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  177 non-null    geometry
dtypes: geometry(1)
memory usage: 6.8+ KB


None

### Сохранение слоя с улицами (Load)

In [122]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
# TODO для того чтобы переименовать индекс, нужно обратиться и нему и вызвать от него метод rename (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.rename.html)
streets = streets.reset_index().rename(columns={"name": "Название улицы"})
streets.index.rename("id", inplace=True)
display(streets)
streets.index

,Название улицы,geometry
id,,
0,1-й Рабфаковский переулок,"LINESTRING (3391743.672 8368417.242, 3391766.1..."
1,1-я дорога,"LINESTRING (3393246.184 8372403.217, 3393268.7..."
2,11-я линия,"LINESTRING (3391094.356 8367013.072, 3391152.4..."
3,13-я линия,"LINESTRING (3391016.210 8366855.634, 3391125.3..."
4,2-й Рабфаковский переулок,"LINESTRING (3392568.994 8367091.715, 3392561.7..."
...,...,...
172,улица Чернова,"MULTILINESTRING ((3391641.514 8369024.600, 339..."
173,улица Чудновского,"MULTILINESTRING ((3392437.637 8382877.735, 339..."
174,улица Шелгунова,"MULTILINESTRING ((3388527.362 8369046.328, 338..."


RangeIndex(start=0, stop=177, step=1, name='id')

In [123]:
# TODO сохранить слой в географической проекции в формате GeoJSON
streets.to_crs(4326).to_file('streets.geojson', driver='GeoJSON')

## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [24]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
gdf_kgiop = gpd.read_file(KGIOP_FILE_URL, mask=district)
display(gdf_kgiop.shape)
display(gdf_kgiop.head(3))
gdf_kgiop.explore(tiles=TILES, height=400)

(249, 9)

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,4072,Палевский жилмассив (19 домов),Жилой дом,1925-1928,"Обуховской Обороны пр., 95, корп. 6",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3387453.441 8376910.050)
1,4051,—,Здание школы (с садом и оградой),1932-1933,"Большой Смоленский пр., 36, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Ансамбль,POINT (3385184.415 8376455.066)
2,3974,—,Октябрьская набережная,1932-1936,"На правом берегу р. Невы, от устья р. Утки до ...",объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Ансамбль,POINT (3388501.848 8377901.347)


### Обработка объектов культурного наследия (Transform)

In [97]:
# TODO добавить два столбца lon и lat, в которых будут долгота и широта
gdf_kgiop["lon"] = gdf_kgiop.geometry.apply(lambda geo: geo.x)
gdf_kgiop["lat"] = gdf_kgiop.geometry.apply(lambda geo: geo.y)
gdf_kgiop.head(3)

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry,lon,lat
0,4072,Палевский жилмассив (19 домов),Жилой дом,1925-1928,"Обуховской Обороны пр., 95, корп. 6",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3387453.441 8376910.050),3.387453e+06,8.376910e+06
1,4051,—,Здание школы (с садом и оградой),1932-1933,"Большой Смоленский пр., 36, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Ансамбль,POINT (3385184.415 8376455.066),3.385184e+06,8.376455e+06
2,3974,—,Октябрьская набережная,1932-1936,"На правом берегу р. Невы, от устья р. Утки до ...",объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Ансамбль,POINT (3388501.848 8377901.347),3.388502e+06,8.377901e+06


### Сохранение слоя с объектами культурного наследия (Load)

In [99]:
gdf_kgiop.shape, gdf_kgiop.object_name.nunique()

((249, 11), 167)

In [103]:
gdf_kgiop[gdf_kgiop.duplicated(subset=["object_name", "object_location", "occurrence_time"])]

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry,lon,lat


In [105]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
columns_rus = {"ensemble_name": "Название комплекса",
               "object_name": "Наименование объекта",
               "occurrence_time": "Время создания",
               "object_location": "Адрес расположения",
               "historical_category": "Категория",
               "normative_act": "Нормативный акт",
               "object_type": "Тип объекта",
               "lon": "Долгота",
               "lat": "Широта"}

gdf_kgiop.rename(columns=columns_rus, inplace=True)
gdf_kgiop.head(3)

,id,Название комплекса,Наименование объекта,Время создания,Адрес расположения,Категория,Нормативный акт,Тип объекта,geometry,Долгота,Широта
0,4072,Палевский жилмассив (19 домов),Жилой дом,1925-1928,"Обуховской Обороны пр., 95, корп. 6",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3387453.441 8376910.050),3.387453e+06,8.376910e+06
1,4051,—,Здание школы (с садом и оградой),1932-1933,"Большой Смоленский пр., 36, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Ансамбль,POINT (3385184.415 8376455.066),3.385184e+06,8.376455e+06
2,3974,—,Октябрьская набережная,1932-1936,"На правом берегу р. Невы, от устья р. Утки до ...",объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Ансамбль,POINT (3388501.848 8377901.347),3.388502e+06,8.377901e+06


In [106]:
# TODO сохранить слой в географической проекции в формате GeoJSON
gdf_kgiop.to_crs(4326).to_file('kgiop.geojson', driver='GeoJSON')

## Визуализация через KeplerGL

In [124]:
map = KeplerGl(height=600)
map.add_data(data=streets.copy(), name="Улицы")
map.add_data(data=gdf_kgiop.copy(), name="Объекты культурного наследия")
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Улицы': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…